# Park a lot

This notebook is put together by the following parts:  
1. A function to segment video frames to locate cars, which will be used to check if the car is going in or out of the parking lot or garage.  
2. A function to read license plate on the segmented frames that will be converted to a string and stored in the 
3. A csv file containing the parking times of the license plates and the size of the car.
4. An additional csv file containing the cars currently in the parking that will be checked against when a license plate apears on the camera.



In [ ]:
# Install requirements
# to run the code, the following are required, and should be installed in a new environment:

# create new environment
%conda create -n "park-3.9" python=3.9 ipython
# activate environment
%conda activate park-3.9
# install package
%conda install numpy
# install requirements.txt
%pip install -r requirements.txt --verbose
# install ipykernel
%conda install -n park-3.9 ipykernel --update-deps --force-reinstall

In [11]:
# import libraries
import argparse
import os

# limit the number of cpus used by high performance libraries
os.environ["OMP_NUM_THREADS"] = "4"
os.environ["OPENBLAS_NUM_THREADS"] = "4"
os.environ["MKL_NUM_THREADS"] = "4"
os.environ["VECLIB_MAXIMUM_THREADS"] = "4"
os.environ["NUMEXPR_NUM_THREADS"] = "4"

import sys
import time
import csv
import glob
import pandas as pd

import numpy as np
from pathlib import Path
import torch
import torch.backends.cudnn as cudnn

In [13]:
# import more libraries ++
ROOT = Path(os.getcwd()).resolve()  # yolov5 strongsort root directory
WEIGHTS = ROOT / 'weights'
print(os.getcwd())

if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
if str(ROOT / 'yolov5') not in sys.path:
    sys.path.append(str(ROOT / 'yolov5'))  # add yolov5 ROOT to PATH
if str(ROOT / 'trackers' / 'strong_sort') not in sys.path:
    sys.path.append(str(ROOT / 'trackers' / 'strong_sort'))  # add strong_sort ROOT to PATH
if str(ROOT / 'trackers' / 'strong_sort' / 'sort') not in sys.path:
    sys.path.append(str(ROOT / 'trackers' / 'strong_sort' / 'sort'))  # add strong_sort ROOT to PATH

import logging
from yolov5.models.common import DetectMultiBackend
from yolov5.utils.dataloaders import VID_FORMATS, LoadImages, LoadStreams
from yolov5.utils.general import (LOGGER, check_img_size, non_max_suppression, scale_boxes, check_requirements, cv2,
                                  check_imshow, xyxy2xywh, increment_path, strip_optimizer, colorstr, print_args, check_file)
from yolov5.utils.torch_utils import select_device, time_sync
from yolov5.utils.plots import Annotator, colors, save_one_box
from trackers.multi_tracker_zoo import create_tracker

# set working directory to this file
os.chdir(ROOT)
print(os.getcwd())

/Users/alex/Library/Mobile Documents/com~apple~CloudDocs/UiA/ikt452_comp_vis/park-it
/Users/alex/Library/Mobile Documents/com~apple~CloudDocs/UiA/ikt452_comp_vis/park-it


In [14]:
# track and sort
@torch.no_grad()
def run(threshold_1=1,
        threshold_2=1,
        smallest_boundingbox_width=1,
        smallest_boundingbox_heigth=1,
        source='0',
        yolo_weights=WEIGHTS / 'yolov5m.pt',  # model.pt path(s),
        reid_weights=WEIGHTS / 'osnet_x0_25_msmt17.pt',  # model.pt path,
        tracking_method='strongsort',
        imgsz=(640, 640),  # inference size (height, width)
        conf_thres=0.25,  # confidence threshold
        iou_thres=0.45,  # NMS IOU threshold
        max_det=1000,  # maximum detections per image
        device='',  # cuda device, i.e. 0 or 0,1,2,3 or cpu
        show_vid=False,  # show results
        save_txt=False,  # save results to *.txt
        save_conf=False,  # save confidences in --save-txt labels
        save_crop=False,  # save cropped prediction boxes
        save_vid=False,  # save confidences in --save-txt labels
        nosave=False,  # do not save images/videos
        classes=None,  # filter by class: --class 0, or --class 0 2 3
        agnostic_nms=False,  # class-agnostic NMS
        augment=False,  # augmented inference
        visualize=False,  # visualize features
        update=False,  # update all models
        project=ROOT / 'runs/track',  # save results to project/name
        name='exp',  # save results to project/name
        exist_ok=False,  # existing project/name ok, do not increment
        line_thickness=2,  # bounding box thickness (pixels)
        hide_labels=False,  # hide labels
        hide_conf=False,  # hide confidences
        hide_class=False,  # hide IDs
        half=False,  # use FP16 half-precision inference
        dnn=False,  # use OpenCV DNN for ONNX inference
        vid_stride=1,  # video frame-rate stride
):

    source = str(source)
    save_img = not nosave and not source.endswith('.txt')  # save inference images
    is_file = Path(source).suffix[1:] in (VID_FORMATS)
    is_url = source.lower().startswith(('rtsp://', 'rtmp://', 'http://', 'https://'))
    webcam = source.isnumeric() or source.endswith('.txt') or (is_url and not is_file)
    if is_url and is_file:
        source = check_file(source)  # download

    # Directories
    if not isinstance(yolo_weights, list):  # single yolo model
        exp_name = yolo_weights.stem
    elif type(yolo_weights) is list and len(yolo_weights) == 1:  # single models after --yolo_weights
        exp_name = Path(yolo_weights[0]).stem
    else:  # multiple models after --yolo_weights
        exp_name = 'ensemble'
    exp_name = name if name else exp_name + "_" + reid_weights.stem
    save_dir = increment_path(Path(project) / exp_name, exist_ok=exist_ok)  # increment run
    (save_dir / 'tracks' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    # Load model
    device = select_device(device)
    model = DetectMultiBackend(yolo_weights, device=device, dnn=dnn, data=None, fp16=half)
    stride, names, pt = model.stride, model.names, model.pt
    imgsz = check_img_size(imgsz, s=stride)  # check image size

    # Dataloader
    if webcam:
        show_vid = check_imshow()
        dataset = LoadStreams(source, img_size=imgsz, stride=stride, auto=pt, vid_stride=vid_stride)
        nr_sources = len(dataset)
    else:
        dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt)
        nr_sources = 1
    vid_path, vid_writer, txt_path = [None] * nr_sources, [None] * nr_sources, [None] * nr_sources

    # Create as many strong sort instances as there are video sources
    tracker_list = []
    for i in range(nr_sources):
        tracker = create_tracker(tracking_method, reid_weights, device, half)
        tracker_list.append(tracker, )
        if hasattr(tracker_list[i], 'model'):
            if hasattr(tracker_list[i].model, 'warmup'):
                tracker_list[i].model.warmup()
    outputs = [None] * nr_sources
    print(outputs)

    # Run tracking
    #model.warmup(imgsz=(1 if pt else nr_sources, 3, *imgsz))  # warmup
    dt, seen = [0.0, 0.0, 0.0, 0.0], 0
    curr_frames, prev_frames = [None] * nr_sources, [None] * nr_sources
    temp_array = ""
    for frame_idx, (path, im, im0s, vid_cap, s) in enumerate(dataset):
        if frame_idx % 5 != 0:
            continue
        t1 = time_sync()
        im = torch.from_numpy(im).to(device)
        im = im.half() if half else im.float()  # uint8 to fp16/32
        im /= 255.0  # 0 - 255 to 0.0 - 1.0
        if len(im.shape) == 3:
            im = im[None]  # expand for batch dim
        t2 = time_sync()
        dt[0] += t2 - t1

        # Inference
        visualize = increment_path(save_dir / Path(path[0]).stem, mkdir=True) if visualize else False
        pred = model(im, augment=augment, visualize=visualize)
        t3 = time_sync()
        dt[1] += t3 - t2

        # Apply NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        dt[2] += time_sync() - t3

        
        # Process detections
        for i, det in enumerate(pred):  # detections per image
            seen += 1
            if webcam:  # nr_sources >= 1
                p, im0, _ = path[i], im0s[i].copy(), dataset.count
                p = Path(p)  # to Path
                s += f'{i}: '
                txt_file_name = p.name
                save_path = str(save_dir / p.name)  # im.jpg, vid.mp4, ...
            else:
                p, im0, _ = path, im0s.copy(), getattr(dataset, 'frame', 0)
                p = Path(p)  # to Path
                # video file
                if source.endswith(VID_FORMATS):
                    txt_file_name = p.stem
                    save_path = str(save_dir / p.name)  # im.jpg, vid.mp4, ...
                # folder with imgs
                else:
                    txt_file_name = p.parent.name  # get folder name containing current img
                    save_path = str(save_dir / p.parent.name)  # im.jpg, vid.mp4, ...
            curr_frames[i] = im0

            txt_path = str(save_dir / 'tracks' / txt_file_name)  # im.txt
            s += '%gx%g ' % im.shape[2:]  # print string
            imc = im0.copy() if save_crop else im0  # for save_crop

            annotator = Annotator(im0, line_width=line_thickness, example=str(names))
            
            if hasattr(tracker_list[i], 'tracker') and hasattr(tracker_list[i].tracker, 'camera_update'):
                if prev_frames[i] is not None and curr_frames[i] is not None:  # camera motion compensation
                    tracker_list[i].tracker.camera_update(prev_frames[i], curr_frames[i])

            if det is not None and len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_boxes(im.shape[2:], det[:, :4], im0.shape).round()  # xyxy

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                # pass detections to strongsort
                t4 = time_sync()
                outputs[i] = tracker_list[i].update(det.cpu(), im0)
                t5 = time_sync()
                dt[3] += t5 - t4

                # draw boxes for visualization
                if len(outputs[i]) > 0:
                    for j, (output, conf) in enumerate(zip(outputs[i], det[:, 4])):


                    # ------------------------------------------------------ SAVE TO FILE  ----------------------------------------------------------
                        bbox_top = output[1]
                        bbox_w = output[2] - output[0]
                        bbox_h = output[3] - output[1]

                        cv2.line(im0, (0, threshold_1), (im0.shape[1], threshold_1), (255, 0, 0), 2) # Blue
                        cv2.line(im0, (0, threshold_2), (im0.shape[1], threshold_2), (255, 0, 0), 2) # Blue

                        if (bbox_top+bbox_h < threshold_1) and (bbox_top+bbox_h > threshold_2) and(bbox_w > smallest_boundingbox_width) and (bbox_h > smallest_boundingbox_heigth):
                            cv2.imwrite(str(save_dir)+"/frame id_"+str(frame_idx+1)+".jpg", im0)


                            bboxes = output[0:4]
                            id = output[4]
                            cls = output[5]
                            if save_txt:
                                # to MOT format
                                bbox_left = output[0]
                                # append to string

                                # add info to array
                                if frame_idx % 100 != 0:
                                    temp_array += f'{frame_idx + 1},{id},{bbox_left},{bbox_top},{bbox_w},{bbox_h},{time.time()}\n'
                                else:
                                    # Write MOT compliant results to file
                                    with open(txt_path + '.csv', 'a') as f:
                                        f.write(temp_array)
                                        f.close()
                                        temp_array = ""

                            if save_vid or save_crop or show_vid:  # Add bbox to image
                                c = int(cls)  # integer class
                                id = int(id)  # integer id
                                label = None if hide_labels else (f'{id} {names[c]}' if hide_conf else \
                                    (f'{id} {conf:.2f}' if hide_class else f'{id} {names[c]} {conf:.2f}'))
                                annotator.box_label(bboxes, label, color=colors(c, True))
                                if save_crop:
                                    txt_file_name = txt_file_name if (isinstance(path, list) and len(path) > 1) else ''
                                    save_one_box(bboxes, imc, file=save_dir / 'crops' / txt_file_name / names[c] / f'{id}' / f'{p.stem}.jpg', BGR=True)

                LOGGER.info(f'{s}Done. yolo:({t3 - t2:.3f}s), {tracking_method}:({t5 - t4:.3f}s)')

            else:
                #strongsort_list[i].increment_ages()
                LOGGER.info('No detections')          

            # Stream results
            im0 = annotator.result()
            if show_vid:
                cv2.imshow(str(p), im0)
                cv2.waitKey(1)  # 1 millisecond

            # Save results (image with detections)
            if save_vid:
                if vid_path[i] != save_path:  # new video
                    vid_path[i] = save_path
                    if isinstance(vid_writer[i], cv2.VideoWriter):
                        vid_writer[i].release()  # release previous video writer
                    if vid_cap:  # video
                        fps = vid_cap.get(cv2.CAP_PROP_FPS)
                        w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                        h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                    else:  # stream
                        fps, w, h = 30, im0.shape[1], im0.shape[0]
                    save_path = str(Path(save_path).with_suffix('.mp4'))  # force *.mp4 suffix on results videos
                    vid_writer[i] = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
                vid_writer[i].write(im0)

            prev_frames[i] = curr_frames[i]

    # Print results
    t = tuple(x / seen * 1E3 for x in dt)  # speeds per image
    LOGGER.info(f'Speed: %.1fms pre-process, %.1fms inference, %.1fms NMS, %.1fms {tracking_method} update per image at shape {(1, 3, *imgsz)}' % t)
    if save_txt or save_vid:
        s = f"\n{len(list(save_dir.glob('tracks/*.txt')))} tracks saved to {save_dir / 'tracks'}" if save_txt else ''
        LOGGER.info(f"Results saved to {colorstr('bold', save_dir)}{s}")
    if update:
        strip_optimizer(yolo_weights)  # update model (to fix SourceChangeWarning)



In [15]:
# set arguments for the track and sort function
def parse_opt(video_path):
    parser = argparse.ArgumentParser()
    parser.add_argument('--yolo-weights', nargs='+', type=Path, default=WEIGHTS / 'yolov5n.pt', help='model.pt path(s)')
    parser.add_argument('--reid-weights', type=Path, default=WEIGHTS / 'osnet_x0_25_msmt17.pt')
    parser.add_argument('--tracking-method', type=str, default='strongsort', help='strongsort, ocsort, bytetrack')
    parser.add_argument('--source', type=str, default='0', help='file/dir/URL/glob, 0 for webcam')  
    parser.add_argument('--imgsz', '--img', '--img-size', nargs='+', type=int, default=[640], help='inference size h,w')
    parser.add_argument('--conf-thres', type=float, default=0.5, help='confidence threshold')
    parser.add_argument('--iou-thres', type=float, default=0.5, help='NMS IoU threshold')
    parser.add_argument('--max-det', type=int, default=1000, help='maximum detections per image')
    parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
    parser.add_argument('--show-vid', action='store_true', help='display tracking video results')
    parser.add_argument('--save-txt', action='store_true', help='save results to *.txt')
    parser.add_argument('--save-conf', action='store_true', help='save confidences in --save-txt labels')
    parser.add_argument('--save-crop', action='store_true', help='save cropped prediction boxes')
    parser.add_argument('--save-vid', action='store_true', help='save video tracking results')
    parser.add_argument('--nosave', action='store_true', help='do not save images/videos')
    # class 0 is person, 1 is bycicle, 2 is car... 79 is oven
    parser.add_argument('--classes', nargs='+', type=int, help='filter by class: --classes 0, or --classes 0 2 3')
    parser.add_argument('--agnostic-nms', action='store_true', help='class-agnostic NMS')
    parser.add_argument('--augment', action='store_true', help='augmented inference')
    parser.add_argument('--visualize', action='store_true', help='visualize features')
    parser.add_argument('--update', action='store_true', help='update all models')
    parser.add_argument('--project', default=ROOT / 'runs/track', help='save results to project/name')
    parser.add_argument('--name', default='exp', help='save results to project/name')
    parser.add_argument('--exist-ok', action='store_true', help='existing project/name ok, do not increment')
    parser.add_argument('--line-thickness', default=2, type=int, help='bounding box thickness (pixels)')
    parser.add_argument('--hide-labels', default=False, action='store_true', help='hide labels')
    parser.add_argument('--hide-conf', default=False, action='store_true', help='hide confidences')
    parser.add_argument('--hide-class', default=False, action='store_true', help='hide IDs')
    parser.add_argument('--half', action='store_true', help='use FP16 half-precision inference')
    parser.add_argument('--dnn', action='store_true', help='use OpenCV DNN for ONNX inference')
    parser.add_argument('--vid-stride', type=int, default=1, help='video frame-rate stride')

    opt = parser.parse_args(['--source', video_path,
                             '--classes', '2',
                             '--save-txt'
                             ,'--show-vid'
                             ])

    # source
    # opt.source = "../park-a-lot/film.mp4"

    # detect only car class
    # opt.classes = 2
    
    # save text file
    # opt.save_txt = True
    # print working directory
    # print(os.getcwd())

    opt.imgsz *= 2 if len(opt.imgsz) == 1 else 1  # expand
    print_args(vars(opt))
    return opt

In [16]:
# Run the track and sort function on the video specified here !!!! ---
def track_cars(video_path, threshold_1,threshold_2, smallest_boundingbox_w, smallest_boundingbox_h):
    print(os.getcwd())
    # set path to video
    opt = parse_opt(video_path)
    # check_requirements(requirements=ROOT / 'requirements.txt', exclude=('tensorboard', 'thop'))
    width = int(cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FRAME_HEIGHT))
    run(**vars(opt), threshold_1=int(height*threshold_1), threshold_2=int(height*threshold_2), smallest_boundingbox_width=smallest_boundingbox_w, smallest_boundingbox_heigth=smallest_boundingbox_h)
    return width, height

# enter threshold values here`
frame_width, frame_height = track_cars(video_path='IMG_0346.mp4', threshold_1=0.98, threshold_2=0.75, smallest_boundingbox_w=150, smallest_boundingbox_h=150)

2019945111: yolo_weights=/Users/alex/Library/Mobile Documents/com~apple~CloudDocs/UiA/ikt452_comp_vis/park-it/weights/yolov5n.pt, reid_weights=/Users/alex/Library/Mobile Documents/com~apple~CloudDocs/UiA/ikt452_comp_vis/park-it/weights/osnet_x0_25_msmt17.pt, tracking_method=strongsort, source=IMG_0346.mp4, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.5, max_det=1000, device=, show_vid=True, save_txt=True, save_conf=False, save_crop=False, save_vid=False, nosave=False, classes=[2], agnostic_nms=False, augment=False, visualize=False, update=False, project=/Users/alex/Library/Mobile Documents/com~apple~CloudDocs/UiA/ikt452_comp_vis/park-it/runs/track, name=exp, exist_ok=False, line_thickness=2, hide_labels=False, hide_conf=False, hide_class=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 2022-12-2 Python-3.9.15 torch-1.13.0 CPU



/Users/alex/Library/Mobile Documents/com~apple~CloudDocs/UiA/ikt452_comp_vis/park-it


100%|██████████| 3.87M/3.87M [00:00<00:00, 13.8MB/s]

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients
No detections


Successfully loaded pretrained weights from "/Users/alex/Library/Mobile Documents/com~apple~CloudDocs/UiA/ikt452_comp_vis/park-it/weights/osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
[None]


No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
No detections
video 1/1 (186/1233) /Users/alex/Library/Mobile Documents/com~apple~CloudDocs/UiA/ikt452_comp_vis/park-it/IMG_0346.mp4: 384x640 1 car, Done. yolo:(0.037s), strongsort:(0.015s)
video 1/1 (191/1233) /Users/alex/Library/Mobile Documents/com~apple~CloudDocs/UiA/ikt452_comp_vis/park-it/IMG_0346.mp4: 384x640 1 car, Done. yolo:(0.038s), strongsort:(0.015s)
video 1/1 (196/1233) /Users/alex/Library/Mobile Documents/com~apple~CloudDocs/UiA/ikt452_comp_vis/park-it/IMG_0346.mp4: 384x640 1 car, Done. yol

In [18]:
frame_width = int(cv2.VideoCapture('./IMG_0346.mp4').get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cv2.VideoCapture('./IMG_0346.mp4').get(cv2.CAP_PROP_FRAME_HEIGHT))
print(os.getcwd())
file_pth = "exp2"
number_of_cars_in_file = 3

def load_data():
    data_file = glob.glob('runs/track/'+file_pth+'/tracks/*.csv')
    with open(data_file[0], 'r') as f:
        data = list(csv.reader(f))
        f.close()
        return  [[float(x) for x in row] for row in data]


def delete_data(all_ids):
    data_file = glob.glob('runs/track/'+file_pth+'/tracks/*.csv')
    keep_rows = list()
    with open(data_file[0], 'r') as f:
        data = list(csv.reader(f))
        for row in data:
            if int(float(row[1])) not in all_ids:
                keep_rows.append(row)

    # write the rows we want to keep to the file
    with open(data_file[0], 'w') as f:
        writer = csv.writer(f)
        writer.writerows(keep_rows)

    f.close()


def sort_into_dict(data):
    dict = {}
    for each_entry in data:
        frame_number = int(each_entry[0])
        car_id = int(each_entry[1])
        bounding_box_and_time = [each_entry[2:6], each_entry[6]]

        # check if id already exists in dict then append the data to the list with the frame number as key
        if car_id in dict:
            dict[car_id][frame_number] = bounding_box_and_time
        else:
            dict[car_id] = {frame_number: bounding_box_and_time}


    # remove the ids that have less than 5 entries because:
    # they might be cars that are on their way in our out of the frame or just noise
    ids_to_remove = []
    for each_id in dict: 
        if len(dict[each_id]) < number_of_cars_in_file:
            ids_to_remove.append(each_id)

    for each_id in ids_to_remove:
        del dict[each_id]

    return dict


def get_best_license_plate(car_dict):
    car_entering = False
    sorting_list = []
    
    for frame_number in car_dict:
        # frame = each[0]
        size = car_dict[frame_number][0][-1] * car_dict[frame_number][0][-2]
        right_side = car_dict[frame_number][0][0] + car_dict[frame_number][0][2]
        left_side = car_dict[frame_number][0][0]
        
        if (left_side > 5) and (right_side < (frame_width-5)):
            sorting_list.append((frame_number, size))

    # sort the list by bounding box size
    sorting_list.sort(key=lambda x: x[1], reverse=True)

    # check if the biggest frame is the first or last frame, which means the car is entering or leaving the frame
    length_of_sort = len(sorting_list)-1
    if sorting_list[0][0] > sorting_list[length_of_sort][0] and sorting_list[0][0] > sorting_list[int(length_of_sort/2)][0]:
        car_entering = True

    # return boolean if car is entering or leaving the parking lot, and the frame numbers of the 3 biggest bounding boxes found
    return car_entering, [x[0] for x in sorting_list[:3]]


def load_image(frame_number, bounding_box_array):
    # load image
    img = cv2.imread('runs/track/'+file_pth+'/frame id_'+str(frame_number)+'.jpg')
    # crop the image to the size of the bounding box
    img = img[int(bounding_box_array[1]):int(bounding_box_array[1]+bounding_box_array[3]), int(bounding_box_array[0]):int(bounding_box_array[0]+bounding_box_array[2])]
    return img


def get_car_images(car_dict):
    car_array = []
    for car_id in car_dict:
        # get the best license plate for each car
        car_entering, frames = get_best_license_plate(car_dict[car_id])
        cropped_car_images = []
        for each_frame in frames:
            img = load_image(each_frame, car_dict[car_id][each_frame][0])
            cropped_car_images.append(img)
        
        car_array.append([car_entering, cropped_car_images])

    return car_array


def delete_images(car_dictionary):
    # frames with multiple cars
    # frames with no cars
    # 

    for car_id in car_dictionary:
        for frame in car_dictionary[car_id]:
            os.remove('runs/track/'+file_pth+'/frame id_'+str(frame)+'.jpg')


# TODO: if frame contains multiple cars, do not delete the image

def fetch_cars():
    # load data from csv file
    data = load_data()

    # sort data into dictionary
    car_dictionary = sort_into_dict(data)

    # delete the loaded data from the csv file
    delete_data(list(car_dictionary.keys()))

    # get the 3 best license plate photos for each car and the direction of the car
    car_array = get_car_images(car_dictionary)
    
    # delete the images containing the cars extracted
    delete_images(car_dictionary)

    return car_array
    

cars_for_peshang = fetch_cars()

image = cars_for_peshang[0][1][0]
cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

/Users/alex/Library/Mobile Documents/com~apple~CloudDocs/UiA/ikt452_comp_vis/park-it
